https://medium.com/@pushkarmandot/build-your-first-deep-learning-neural-network-model-using-keras-in-python-a90b5864116d
#Data and Business Problem:
#Our basic aim is to predict customer churn for a certain bank i.e. which customer is going to leave this bank service.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('Churn_Modelling.csv')

dataset.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Create matrix of features and matrix of target variable. In this case we are excluding column 1, 2 & 3 as those are ‘row_number’, ‘customerid’ & 'Surname' which are not useful in our analysis. Column 14, ‘Exited’ is our Target Variable

In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
## Read this for categorical Encoding : http://pbpython.com/categorical-encoding.html
##pd.get_dummies(dataset, columns=["Geography", "Gender"], prefix=["Geography", "Gender"]).head()

In [3]:
def getXy_1(dataset,target):
    df = pd.get_dummies(dataset, columns=["Geography", "Gender"], prefix=["Geography", "Gender"])
    
    y = df[target]
    X = df.loc[:, df.columns != target]
    return X,y


def getXy_2(dataset,target):    
    lb = LabelEncoder()
    dataset['Gender'] = lb.fit_transform(dataset['Gender'])
    dataset['Geography'] = lb.fit_transform(dataset['Geography'])
    ## One-Hot Coding
    dataset = pd.get_dummies(dataset, columns = ['Geography','Gender'])
    y = dataset[target]
    X = dataset.loc[:, dataset.columns != target]
    return X,y
    

In [4]:
X,y = getXy_2(dataset,target='Exited')
print("X.columns:",X.columns)
X = X.iloc[:, 3:].values
y = y.values
y

X.columns: Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Geography_0', 'Geography_1', 'Geography_2',
       'Gender_0', 'Gender_1'],
      dtype='object')


array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

 I know you are tired of data preprocessing but I promise this is the last step. If you carefully observe data, you will find that data is not scaled properly. Some variable has value in thousands while some have value is tens or ones. We don’t want any of our variable to dominate on other so let’s go and scale data.
 
 ‘StandardScaler’ is available in ScikitLearn. In the following code we are fitting and transforming StandardScaler method on train data. We have to standardize our scaling so we will use the same fitted method to transform/scale test data.

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [7]:
X_train

array([[ 0.4034289 ,  0.85614078, -1.03419226, ...,  1.73205081,
         1.09196221, -1.09196221],
       [ 1.21234527, -1.04800752, -0.6883953 , ..., -0.57735027,
         1.09196221, -1.09196221],
       [ 0.33083384, -0.57197045,  1.38638647, ..., -0.57735027,
         1.09196221, -1.09196221],
       ..., 
       [ 0.78714564, -0.38155562,  0.00319862, ..., -0.57735027,
        -0.91578261,  0.91578261],
       [-0.85142856,  0.3801037 ,  1.04058951, ...,  1.73205081,
        -0.91578261,  0.91578261],
       [-0.24992664, -0.38155562,  0.69479255, ...,  1.73205081,
         1.09196221, -1.09196221]])

In [8]:
X_train.shape

(8000, 13)

In [9]:
import keras
from keras.models import Sequential

from keras.layers.core import Dense,Dropout, Dropout,Activation

Using TensorFlow backend.


In [10]:
model = Sequential()

In [11]:
model.add(Dense(16,input_dim=13))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(8))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                224       
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
__________

In [13]:
# Compiling Neural Network
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [14]:
# Fitting our model 
model.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

C:\Users\prassha\AppData\Local\Continuum\Anaconda3\lib\site-packages\keras\models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
8000/8000 [==============================] - 1s - loss: 0.5072 - acc: 0.7852     
Epoch 2/100
8000/8000 [==============================] - 1s - loss: 0.4530 - acc: 0.8054     - ETA: 0s - loss: 0.46
Epoch 3/100
8000/8000 [==============================] - 1s - loss: 0.4448 - acc: 0.8094     - ETA: 0s - loss: 0.4504 - acc:
Epoch 4/100
8000/8000 [==============================] - 1s - loss: 0.4370 - acc: 0.8106     
Epoch 5/100
8000/8000 [==============================] - 1s - loss: 0.4254 - acc: 0.8161     
Epoch 6/100
8000/8000 [==============================] - 1s - loss: 0.4220 - acc: 0.8200     - ETA: 0s - loss: 0.4213 - acc: 0.8
Epoch 7/100
8000/8000 [==============================] - 1s - loss: 0.4067 - acc: 0.8296     
Epoch 8/100
8000/8000 [==============================] - 1s - loss: 0.4011 - acc: 0.8320     - ETA: 0s - loss: 0.4032 - acc: 0.83 - ETA: 0s - loss: 0.3954
Epoch 9/100
8000/8000 [==============================] - 1s - loss: 0.3975 - acc: 0.8350     
Epoch

8000/8000 [==============================] - 1s - loss: 0.3574 - acc: 0.8522     
Epoch 82/100
8000/8000 [==============================] - 1s - loss: 0.3544 - acc: 0.8545     
Epoch 83/100
8000/8000 [==============================] - 1s - loss: 0.3556 - acc: 0.8539     
Epoch 84/100
8000/8000 [==============================] - 1s - loss: 0.3573 - acc: 0.8531     
Epoch 85/100
8000/8000 [==============================] - 1s - loss: 0.3594 - acc: 0.8516     
Epoch 86/100
8000/8000 [==============================] - 1s - loss: 0.3576 - acc: 0.8550     
Epoch 87/100
8000/8000 [==============================] - 1s - loss: 0.3557 - acc: 0.8516     
Epoch 88/100
8000/8000 [==============================] - 1s - loss: 0.3540 - acc: 0.8551     
Epoch 89/100
8000/8000 [==============================] - 1s - loss: 0.3593 - acc: 0.8477     
Epoch 90/100
8000/8000 [==============================] - 1s - loss: 0.3544 - acc: 0.8499     
Epoch 91/100
8000/8000 [==============================] - 1s - 

In [15]:
##Predicting the test set results
y_pred = model.predict(X_test)
y_pred = (y_pred>0.4)
y_pred

array([[False],
       [False],
       [ True],
       ..., 
       [False],
       [False],
       [False]], dtype=bool)

In [16]:
# Creating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [17]:
cm

array([[1497,  104],
       [ 172,  227]])